In [ ]:
import os
import re
import time
import torch
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

urlPtBr = 'https://raw.githubusercontent.com/CleissonVieira/fake-reviews-bert-ptbr/main/datasets/yelp-fake-reviews-dataset-pt.csv'
urlEn = 'https://raw.githubusercontent.com/CleissonVieira/fake-reviews-bert-ptbr/main/datasets/yelp-fake-reviews-dataset-en.csv'

TOKENIZADOR = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
MODELO_UTILIZADO = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=2)

In [ ]:
# CARREGANDO OS DADOS EM PORTUGUES

dfPtBr = pd.read_csv(urlPtBr)
dfPtBr = dfPtBr[['content', 'fake_review']]

dfReal = dfPtBr[dfPtBr.fake_review == False]
dfFake = dfPtBr[dfPtBr.fake_review == True]

df_balanceado_ptBr = pd.concat([dfReal.sample(n=3387, random_state=42), dfFake.sample(n=3387, random_state=42)]) 
df_balanceado_ptBr['fake_review'] = df_balanceado_ptBr['fake_review'].astype(int)

df_desbalanceado_ptBr = pd.concat([dfReal, dfFake]) 
df_desbalanceado_ptBr['fake_review'] = df_desbalanceado_ptBr['fake_review'].astype(int)


In [ ]:
# CARREGANDO OS DADOS EM INGLÊS

dfEn = pd.read_csv(urlEn)
dfEn = dfEn[['content', 'fake_review']]

dfReal = dfEn[dfEn.fake_review == False]
dfFake = dfEn[dfEn.fake_review == True]

df_balanceado1_en = pd.concat([dfReal.sample(n=3387, random_state=42), dfFake.sample(n=3387, random_state=42)]) 
df_balanceado1_en['fake_review'] = df_balanceado1_en['fake_review'].astype(int)

df_balanceado2_en = pd.concat([dfReal.sample(n=14567, random_state=42), dfFake.sample(n=14567, random_state=42)]) 
df_balanceado2_en['fake_review'] = df_balanceado2_en['fake_review'].astype(int)

df_desbalanceado_en = pd.concat([dfReal.sample(n=3387, random_state=42), dfFake.sample(n=3387, random_state=42)]) 
df_desbalanceado_en['fake_review'] = df_desbalanceado_en['fake_review'].astype(int)

In [ ]:
# VERIFICAR AS FREQUENCIAS DOS TAMANHOS DE TOKENS DAS REVISÕES PARA DEFINIR MAX LENGTH PARA TOKENIZADOR
# IDENTIFICADO QUE 256 E 512 ABRANGE A MAIORIA

dfMaxLength = pd.read_csv(urlPtBr)
#dfMaxLength = pd.read_csv(urlEn)

dfMaxLength['review_length'] = dfMaxLength['content'].apply(len)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
dfMaxLength['num_tokens'] = dfMaxLength['content'].apply(lambda x: len(tokenizer.tokenize(x)))

plt.hist(dfMaxLength['num_tokens'], bins=50)
plt.xlabel('Número de Tokens')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# MÉTODOS PARA RESULTADOS

def GerarMatrizConfusao(y_true, y_pred, class_names, nomeDf, nomePng):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.savefig(f'{PATH_DATA_EXECUCAO}{nomeDf}/{nomePng}.png', format='png')
    #plt.show()

In [ ]:
# CLASSES/MÉTODOS PARA CONFIGURAR/TREINAR/VALIDAR

def TokenizarTextos(texts, labels):
    inputs = TOKENIZADOR(texts.tolist(), max_length=MAX_LENGTH, padding=True, truncation=True, return_tensors="pt")
    inputs['labels'] = torch.tensor(labels)
    return inputs

class DatasetBert(Dataset):
    def __init__(self, encodings): self.encodings = encodings
    def __getitem__(self, idx): return {key: val[idx] for key, val in self.encodings.items()}
    def __len__(self): return len(self.encodings['input_ids'])

def ConfigurarOtimizadorPerdaAgendador(model, train_loader):
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    return optimizer, scheduler

def TreinarModelo(modelo, train_loader, otimizador, agendador, device):
    print('Device utilizado: ', device)
    print(f"Init Treino: Memória GPU alocada: {torch.cuda.memory_allocated()/1024/1024} MB")
    modelo.train()
    perdaTotal = 0

    for batch in train_loader:
        otimizador.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        saidas = modelo(**batch)
        perda = saidas.loss
        perdaTotal += perda.item()
        perda.backward()
        otimizador.step()
        agendador.step()

    print(f"End Treino: Memória GPU alocada: {torch.cuda.memory_allocated()/1024/1024} MB")
    return perdaTotal / len(train_loader)

def AvaliarModelo(modelo, valLoader, nomeDf, nomePng, device):
    modelo.eval()
    preds, trueLabels = [], []

    with torch.no_grad():
        for batch in valLoader:
            batch = {k: v.to(device) for k, v in batch.items()}
            saidas = modelo(**batch)
            logits = saidas.logits
            preds.extend(torch.argmax(logits, axis=-1).cpu().numpy())
            trueLabels.extend(batch['labels'].cpu().numpy())

    #print("\nClassification Report:")
    #print(classification_report(trueLabels, preds, target_names=['Real', 'Fake']))

    report = classification_report(trueLabels, preds, target_names=['Real', 'Fake'], output_dict=True)
    dfClassificationReport = pd.DataFrame(report).transpose()
    dfTitulo = pd.DataFrame([[f'Classification Report ({nomeDf} - {datetime.now().strftime("%Y-%m-%d_%H-%M")}):']])
    file_exists = os.path.isfile(f'{PATH_DATA_EXECUCAO}relatorioMatrizConfusao.csv')
    
    with open(f'{PATH_DATA_EXECUCAO}relatorioMatrizConfusao.csv', 'a') as f:
        dfTitulo.to_csv(f, index=False, header=not file_exists)
        dfClassificationReport.to_csv(f, index=True, header=True) 
        f.write('\n')
    
    GerarMatrizConfusao(trueLabels, preds, ['Real', 'Fake'], nomeDf, nomePng)

    accuracy = accuracy_score(trueLabels, preds)
    precision = precision_score(trueLabels, preds)
    recall = recall_score(trueLabels, preds)
    f1 = f1_score(trueLabels, preds)

    precision_real = precision_score(trueLabels, preds, pos_label=0)
    precision_fake = precision_score(trueLabels, preds, pos_label=1)
    recall_real = recall_score(trueLabels, preds, pos_label=0)
    recall_fake = recall_score(trueLabels, preds, pos_label=1)
    f1_real = f1_score(trueLabels, preds, pos_label=0)
    f1_fake = f1_score(trueLabels, preds, pos_label=1)

    return accuracy, precision, recall, f1, precision_real, precision_fake, recall_real, recall_fake, f1_real, f1_fake

In [ ]:
def IniciarProcesso(dfUtilizado, conjuntoDados, nomeDf, device):
    print(f'\nLR {LEARNING_RATE} | EP {EPOCHS} | {conjuntoDados} | Verdadeiros: {len(dfUtilizado[dfUtilizado.fake_review == False])} | Falsos: {len(dfUtilizado[dfUtilizado.fake_review == True])}')
    if not os.path.exists(f'{PATH_DATA_EXECUCAO}{nomeDf}/'): os.makedirs(f'{PATH_DATA_EXECUCAO}{nomeDf}/')

    dataStart = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    temporizador = time.time()
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    metricas = {
        'Precision_Real': [], 'Precision_Fake': [], 'Precision': [],
        'F1_Score_Real': [], 'F1_Score_Fake': [], 'F1_Score': [],
        'Recall_Real': [], 'Recall_Fake': [], 'Recall': [],
        'Accuracy': [], 'tempoTreino': 0, 'tempoValidacao': 0
    }
    
    conteudoAvaliacao = dfUtilizado['content'].values
    conteudoRealFake = dfUtilizado['fake_review'].values
    
    for dobras, (treinoIndex, testeIndex) in enumerate(skf.split(conteudoAvaliacao, conteudoRealFake)):
        print(f"\nDobras {dobras + 1}/5")
    
        conteudoAvaliacaoTreino, conteudoAvaliacaoTeste = conteudoAvaliacao[treinoIndex], conteudoAvaliacao[testeIndex]
        conteudoRealFakeTreino, conteudoRealFakeTeste = conteudoRealFake[treinoIndex], conteudoRealFake[testeIndex]
    
        treinoEncodings = TokenizarTextos(conteudoAvaliacaoTreino, conteudoRealFakeTreino)
        validacaoEncodings = TokenizarTextos(conteudoAvaliacaoTeste, conteudoRealFakeTeste)
    
        treinoDataset = DatasetBert(treinoEncodings)
        validacaoDataset = DatasetBert(validacaoEncodings)
    
        treinoDataLoader = DataLoader(treinoDataset, batch_size=BATCH_SIZE, shuffle=True)
        validacaoDataLoader = DataLoader(validacaoDataset, batch_size=BATCH_SIZE)
    
        otimizador, agendador = ConfigurarOtimizadorPerdaAgendador(MODELO_UTILIZADO, treinoDataLoader)
    
        temporizador = time.time()
        for epoch in range(EPOCHS):
            perdaTreino = TreinarModelo(MODELO_UTILIZADO, treinoDataLoader, otimizador, agendador, device)
            print(f"Epoch {epoch + 1}, Loss: {perdaTreino:.4f}")
        tempoTreino = time.time() - temporizador

        nomePng = f'{dobras}d_real{len(dfUtilizado[dfUtilizado.fake_review == False])}_fake{len(dfUtilizado[dfUtilizado.fake_review == True])}'
        
        temporizador = time.time()
        accuracy, precision, recall, f1, precisionReal, precisionFake, recallReal, recallFake, f1Real, f1Fake = AvaliarModelo(MODELO_UTILIZADO, validacaoDataLoader, nomeDf, nomePng, device)
        tempoValidacao = time.time() - temporizador
        
        metricas['tempoTreino'] += tempoTreino
        metricas['tempoValidacao'] += tempoValidacao
        metricas['Precision_Real'].append(precisionReal
        metricas['Precision_Fake'].append(precisionFake)
        metricas['Recall_Real'].append(recallReal)
        metricas['Recall_Fake'].append(recallFake)
        metricas['F1_Score_Real'].append(f1Real)
        metricas['F1_Score_Fake'].append(f1Fake)
        metricas['Precision'].append(precision)
        metricas['Recall'].append(recall)
        metricas['F1_Score'].append(f1)
        metricas['Accuracy'].append(accuracy)
        
    return pd.DataFrame([{
        'data_execucao': dataStart,
        'dataset': conjuntoDados,
        'epocas': EPOCHS,
        'max_length_tokenizer': MAX_LENGTH,
        'learning_rate': LEARNING_RATE,
        'tempo_treino_segundos': metricas['tempoTreino'],
        'tempo_validacao_segundos': metricas['tempoValidacao'],
        'total_avaliacoes_dataLoader': len(dfUtilizado),
        'total_avaliacoes_verdadeiras': len(dfUtilizado[dfUtilizado.fake_review == False]),
        'total_avaliacoes_falsas': len(dfUtilizado[dfUtilizado.fake_review == True]),
        'scenario': 'Review',
        'classifier': 'bert-base-multilingual-uncased',
        'features_used': 'content',
        'precision_real': np.mean(metricas['Precision_Real']),
        'precision_fake': np.mean(metricas['Precision_Fake']),
        'precision': np.mean(metricas['Precision']),
        'precision_variance': np.var(metricas['Precision'], ddof=1),
        'precision_min': np.min(metricas['Precision']),
        'precision_max': np.max(metricas['Precision']),
        'f1_score_real': np.mean(metricas['F1_Score_Real']),
        'f1_score_fake': np.mean(metricas['F1_Score_Fake']),
        'f1_score': np.mean(metricas['F1_Score']),
        'f1_score_variance': np.var(metricas['F1_Score'], ddof=1),
        'f1_score_min': np.min(metricas['F1_Score']),
        'f1_score_max': np.max(metricas['F1_Score']),
        'recall_real': np.mean(metricas['Recall_Real']),
        'recall_fake': np.mean(metricas['Recall_Fake']),
        'recall': np.mean(metricas['Recall']),
        'recall_variance': np.var(metricas['Recall'], ddof=1),
        'recall_min': np.min(metricas['Recall']),
        'recall_max': np.max(metricas['Recall']),
        'accuracy': np.mean(metricas['Accuracy']),
        'accuracy_variance': np.var(metricas['Accuracy'], ddof=1),
        'accuracy_min': np.min(metricas['Accuracy']),
        'accuracy_max': np.max(metricas['Accuracy'])
    }]).round(5)

In [ ]:
# EXECUÇÃO PARA TESTES

df_teste1 = pd.concat([dfReal.sample(n=10, random_state=42), dfFake.sample(n=10, random_state=42)]) 
df_teste1['fake_review'] = df_teste1['fake_review'].astype(int)
df_teste2 = pd.concat([dfReal.sample(n=15, random_state=42), dfFake.sample(n=15, random_state=42)]) 
df_teste2['fake_review'] = df_teste2['fake_review'].astype(int)

EPOCHS = 2
BATCH_SIZE = 32
MAX_LENGTH = 256
LEARNING_RATE = 2e-5
PATH_DATA_EXECUCAO = F'../resultados/{datetime.now().strftime("%Y-%m-%d_%H-%M")}/'
RESULTADOS_CSV = F'{PATH_DATA_EXECUCAO}testeResultadoTreinoValidacao.csv'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ' ,'cuda' if torch.cuda.is_available() else 'cpu')
MODELO_UTILIZADO.to(device)

for lr in [1e-5, 2e-5, 3e-5, 5e-5]:
    LEARNING_RATE = lr
    if os.path.isfile(RESULTADOS_CSV):
        IniciarProcesso(df_teste1, 'testando (0)', f'LR_{LEARNING_RATE}_teste1', device).to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)
    else:
        IniciarProcesso(df_teste1, 'testando (0)', f'LR_{LEARNING_RATE}_teste1', device).to_csv(RESULTADOS_CSV, mode='w', index=False)
    
    IniciarProcesso(df_teste2, 'testando (1)', f'LR_{LEARNING_RATE}_teste2', device).to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)

print(f'Resultados armazenados no arquivo {RESULTADOS_CSV}')

In [ ]:
EPOCHS = 5
BATCH_SIZE = 32
MAX_LENGTH = 256
LEARNING_RATE = 2e-5
PATH_DATA_EXECUCAO = F'../resultados/{datetime.now().strftime("%Y-%m-%d_%H-%M")}/'
RESULTADOS_CSV = F'{PATH_DATA_EXECUCAO}resultadosTreinoValidacao.csv'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ' ,'cuda' if torch.cuda.is_available() else 'cpu')
MODELO_UTILIZADO.to(device)

for lr in [1e-5, 2e-5, 3e-5, 5e-5]:
    LEARNING_RATE = lr

    for ep in [5, 10, 15, 20, 25]:
        EPOCHS = ep
        
        if os.path.isfile(RESULTADOS_CSV):
            TreinarValidarModelo(df_balanceado_ptBr, 'Português Balanceado (i)', f'LR_{LEARNING_RATE}_EP_{EPOCHS}_df_balanceado_ptBr', device).to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)
        else:
            TreinarValidarModelo(df_balanceado_ptBr, 'Português Balanceado (i)', f'LR_{LEARNING_RATE}_EP_{EPOCHS}_df_balanceado_ptBr', device).to_csv(RESULTADOS_CSV, mode='w', index=False)
        
        #TreinarValidarModelo(df_balanceado1_en, 'Inglês Balanceado (ii)', 'df_balanceado1_en').to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)
        #TreinarValidarModelo(df_balanceado2_en, 'Inglês Balanceado (iii)', 'df_balanceado2_en').to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)
        TreinarValidarModelo(df_desbalanceado_ptBr, 'Português Desbalanceado (iv)', f'LR_{LEARNING_RATE}_EP_{EPOCHS}_df_desbalanceado_ptBr', device).to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)
        #TreinarValidarModelo(df_desbalanceado_en, 'Inglês Desbalanceado (v)', 'df_desbalanceado_en').to_csv(RESULTADOS_CSV, mode='a', header=False, index=False)

print(f'\nResultados armazenados no arquivo {RESULTADOS_CSV}')

Device:  cuda

LR 1e-05 | EP 5 | Português Balanceado (i) | Verdadeiros: 3387 | Falsos: 3387

Dobras 1/5
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 638.73583984375 MB
End Treino: Memória GPU alocada: 2572.12548828125 MB
Epoch 1, Loss: 0.6123
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 2572.0595703125 MB
End Treino: Memória GPU alocada: 2572.12548828125 MB
Epoch 2, Loss: 0.5584
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 2572.0595703125 MB
End Treino: Memória GPU alocada: 2572.12548828125 MB
Epoch 3, Loss: 0.5145
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 2572.0595703125 MB
End Treino: Memória GPU alocada: 2572.12548828125 MB
Epoch 4, Loss: 0.4578
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 2572.0595703125 MB
End Treino: Memória GPU alocada: 2572.12548828125 MB
Epoch 5, Loss: 0.4117

Dobras 2/5
Device utilizado:  cuda
Init Treino: Memória GPU alocada: 1295.21875 MB
End Treino: Memória GPU alocada: 2573.62548828125 M